In [1]:
import busio ##black sensor library
import board ##black sensor library
import adafruit_ccs811 ##black sensor library
import smbus ##blue sensor libarary
import gspread ##google sheets library
import time ##library for general use

from datetime import datetime ##google sheets import

In [ ]:
from oauth2client.service_account import ServiceAccountCredentials

sheet_name = "EnviroPiLog"

# use creds to create a client to interact with the Google Drive API
scopes = ["https://www.googleapis.com/auth/drive", "https://www.googleapis.com/auth/drive.file", 
          "https://www.googleapis.com/auth/spreadsheets"]
creds = ServiceAccountCredentials.from_json_keyfile_name('client_secret.json', scopes)
client = gspread.authorize(creds)

# Find the workbook by name and open the first sheet
sheet = client.open(sheet_name).sheet1